# 자산양수도 크롤러
## 합병등종료보고서(자산양수도)


### columns

LIST_DT: 상장기일, 등기일  
EVT_DT: 이벤트기일  
  
ORD: 순번  
  
STK_CD : 결과기업 코드  
STK_NM_KOR : 결과기업 한글명  
  
CAPCHG_TYP : 이벤트 타입  
ISSUE_DESCRIPT : 이벤트 한글명  
  
OBJ_STK_CD : 대상기업 코드  
OBJ_STK_NM_KOR : 대상기업 한글명  


양수대금






In [54]:
#-*- coding:utf-8 -*-
# Parsing dividends data from DART
import urllib.request
import urllib.parse
import xlsxwriter
import os
import time
import sys
import getopt
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import re
from urllib.request import urlopen
import parser
import pandas as pd
from collections import defaultdict



# 목록 데이터 리딩 

In [3]:
# 자산양수도= pd.read_excel('C:\\Users\\HS\\Desktop\\3,4차 기업이벤트 양수도 전체 목록(결정 삭제).xlsx')
자산양수도= pd.read_csv('C:\\Users\\HS\\Desktop\\3,4차 기업이벤트 양수도 전체 목록(결정 삭제).csv', encoding='ANSI')
자산양수도.head()


,삭제,bsn_tp,crp_cls,crp_nm,crp_cd,rpt_nm,rcp_no,flr_nm,rcp_dt
0,NaN,E003,Y,동화약품,A000020,합병등종료보고서(자산양수도),20171113000213,동화약품,20171113
1,NaN,E003,Y,KR모터스,A000040,자산양수ㆍ도종료보고서,20070928000333,KR모터스,20070928
2,NaN,E003,Y,CJ대한통운,A000120,[기재정정]영업양수ㆍ도종료보고서,20090122000008,CJ대한통운,20090122
3,NaN,E003,Y,CJ대한통운,A000120,합병등종료보고서(자산양수도),20120102000266,CJ대한통운,20120102
4,NaN,E003,Y,하이트진로홀딩스,A000140,합병등종료보고서(자산양수도),20090410000957,하이트진로홀딩스,20090410


In [4]:
자산양수도['url'] = 자산양수도.rcp_no.apply(lambda x: "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=" + str(x))
자산양수도['기재정정']= 자산양수도['rpt_nm'].isin(['[기재정정]자산양수ㆍ도종료보고서','[기재정정]합병등종료보고서(자산양수도)', '[기재정정]합병등종료보고서(영업양수도)', '[기재정정]영업양수ㆍ도종료보고서']).apply(int)
자산양수도.head()

,삭제,bsn_tp,crp_cls,crp_nm,crp_cd,rpt_nm,rcp_no,flr_nm,rcp_dt,url,기재정정
0,NaN,E003,Y,동화약품,A000020,합병등종료보고서(자산양수도),20171113000213,동화약품,20171113,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,0
1,NaN,E003,Y,KR모터스,A000040,자산양수ㆍ도종료보고서,20070928000333,KR모터스,20070928,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,0
2,NaN,E003,Y,CJ대한통운,A000120,[기재정정]영업양수ㆍ도종료보고서,20090122000008,CJ대한통운,20090122,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,1
3,NaN,E003,Y,CJ대한통운,A000120,합병등종료보고서(자산양수도),20120102000266,CJ대한통운,20120102,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,0
4,NaN,E003,Y,하이트진로홀딩스,A000140,합병등종료보고서(자산양수도),20090410000957,하이트진로홀딩스,20090410,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,0


In [5]:
자산양수도['url'].iloc[3]

'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20120102000266'

In [6]:
자산양수도.rpt_nm.value_counts()

rpt_nm_list= [
    ['합병등종료보고서(자산양수도)','[기재정정]합병등종료보고서(자산양수도)'],  # 8,9
    ['합병등종료보고서(영업양수도)','[기재정정]합병등종료보고서(영업양수도)'],  # 8,9
    ['영업양수ㆍ도종료보고서','[기재정정]영업양수ㆍ도종료보고서'],  # 4,5
    ['자산양수ㆍ도종료보고서','[기재정정]자산양수ㆍ도종료보고서']   # 3,4
    ]

In [7]:
df_sample = 자산양수도.loc[자산양수도.삭제.isnull()].loc[자산양수도.rpt_nm.isin(rpt_nm_list[0])].head()
df_sample
# 자산양수도.loc[자산양수도.삭제.isnull()].loc[자산양수도.rpt_nm.isin(rpt_nm_list[1])]
# 자산양수도.loc[자산양수도.삭제.isnull()].loc[자산양수도.rpt_nm.isin(rpt_nm_list[2])]
# 자산양수도.loc[자산양수도.삭제.isnull()].loc[자산양수도.rpt_nm.isin(rpt_nm_list[3])]

,삭제,bsn_tp,crp_cls,crp_nm,crp_cd,rpt_nm,rcp_no,flr_nm,rcp_dt,url,기재정정
0,NaN,E003,Y,동화약품,A000020,합병등종료보고서(자산양수도),20171113000213,동화약품,20171113,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,0
3,NaN,E003,Y,CJ대한통운,A000120,합병등종료보고서(자산양수도),20120102000266,CJ대한통운,20120102,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,0
4,NaN,E003,Y,하이트진로홀딩스,A000140,합병등종료보고서(자산양수도),20090410000957,하이트진로홀딩스,20090410,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,0
5,NaN,E003,Y,하이트진로홀딩스,A000140,합병등종료보고서(자산양수도),20090521000091,하이트진로홀딩스,20090521,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,0
6,NaN,E003,Y,하이트진로홀딩스,A000140,합병등종료보고서(자산양수도),20090624000313,하이트진로홀딩스,20090625,http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20...,0


In [90]:
# 돌리기~

dart_div_list = list()

for i in range(0,2):
    target = df_sample.iloc[i]
    #print(target)
    crpCd = target.crp_cd
    crpNm = target.crp_nm
    rcpNo = target.rcp_no
    rcpDt = target.rcp_dt
    link = target.url 
    
    handle = urllib.request.urlopen(link)
    print('여기다!!!',handle)
    data = handle.read()
    soup2 = BeautifulSoup(data, 'html.parser', from_encoding='utf-8')  # http read
    #print(soup2)

    test = soup2.find('a', {'href' : '#download'})['onclick']
    #print(test)

    words = test.split("'")
    #print(words)

    rcpNo= words[1]
    dcmNo= words[3]
    #print(rcpNo)
    #print(dcmNo)

    dart2 = soup2.find_all(string=re.compile('dart2.dtd'))
    dart3 = soup2.find_all(string=re.compile('dart3.xsd'))
    #print(dart3)

    if target.기재정정 == 0:
        eleId = '8'  # 재무제표 8
    else: 
        eleId = '9'  # 재무제표 8

    # 뷰어 링크
    if len(dart3) != 0:
        link2 = "http://dart.fss.or.kr/report/viewer.do?rcpNo=" + rcpNo + "&dcmNo=" + dcmNo + "&eleId=" + eleId + "&offset=4916&length=3668&dtd=dart3.xsd"
    print(link2)
    
    ####### 테이블
    handle = urllib.request.urlopen(link2)
    #print(handle)
    data = handle.read()
    soup3 = BeautifulSoup(data, 'html.parser', from_encoding='utf-8')
    #print(soup3)

    tables = soup3.findAll("table")
    print('len(tables):',len(tables))  # list
#     print(tables)  # list
      
        
    if len(tables) == 1 :  # 재무제표만
        div_table = soup3.find("table")
    elif len(tables)== 2:  # 단위, 재무제표   
        div_table = soup3.findAll("table")[-1]    
#     else:
#         print("Tables", len(tables))
#         div_table = soup3.findAll("table")[2]
#         div_table = soup3.findAll("table")[-1]



#     print(div_table)  # 예외처리 후 한가지의 table 파싱
    
    div_trs = div_table.findAll('tr')
    print(len(div_trs))
#     print(div_trs)
    
    
    dict1= defaultdict(list)
    
    
    # 딕셔너리에 넣기
    for i in range(len(div_trs)):
        for j in range(1,4):
            try:
                key = cleansing(div_trs[i].findAll('td')[0].text.strip())
                val = cleansing(div_trs[i].findAll('td')[j].text.strip())
                dict1[key].append(val)
            except IndexError:
                pass
    print(dict(dict1))
    dict1['STK_CD'].append(crpCd)
    dict1['STK_NM_KOR'].append(crpNm)
    dict1['rcp_NO'].append(rcpNo)
    dict1['rcp_DT'].append(rcpDt)
    dict1['url'].append(link)
    
    dart_div_list.append(dict(dict1))
    
    
print("")
print("")
print('dart_div_list')
print(dart_div_list)


여기다!!! <http.client.HTTPResponse object at 0x0000017B45BB2A58>
http://dart.fss.or.kr/report/viewer.do?rcpNo=20171113000213&dcmNo=5840458&eleId=8&offset=4916&length=3668&dtd=dart3.xsd
len(tables): 1
14
{'구분': ['자산양수도전', '증가감소', '자산양수도후'], '자산': ['', '', ''], '유동자산': ['162257', '85350', '247607'], '비유동자산': ['170261', '30450', '139811'], '자산총계': ['332518', '54900', '387418'], '부채': ['', '', ''], '유동부채': ['57216', '350', '57566'], '비유동부채': ['18546', '', '18546'], '부채총계': ['75762', '350', '76112'], '자본': ['', '', ''], '자본금': ['27931', '', '27931'], '기타자본항목': ['228825', '54550', '283375'], '자본총계': ['256756', '54550', '311306'], '자산및부채총계': ['332518', '54900', '387418']}
여기다!!! <http.client.HTTPResponse object at 0x0000017B45C69F28>
http://dart.fss.or.kr/report/viewer.do?rcpNo=20120102000266&dcmNo=3235174&eleId=8&offset=4916&length=3668&dtd=dart3.xsd
len(tables): 2
27
{'유동자산': ['809433', '41633', '851066'], '당좌자산': ['400638', '361500', '762128'], '재고자산': ['8928', '', '8928'], '매각예정비유동자산': ['31

In [95]:
cur_dir = os.getcwd()
cur_dir

workbook_name = "DART_dividends_baseline.xlsx"

workbook = xlsxwriter.Workbook(workbook_name)

worksheet_result = workbook.add_worksheet('result')
filter_format = workbook.add_format({'bold':True,
                                    'fg_color': '#D7E4BC'
                                    })

percent_format = workbook.add_format({'num_format': '0.00%'})

roe_format = workbook.add_format({'bold':True,
                                  'underline': True,
                                  'num_format': '0.00%'})

num_format = workbook.add_format({'num_format':'0.00'})
num2_format = workbook.add_format({'num_format':'#,##0'})
num3_format = workbook.add_format({'num_format':'#,##0.00',
                                  'fg_color':'#FCE4D6'})

worksheet_result.set_column('A:A', 10)
worksheet_result.set_column('B:B', 10)
worksheet_result.set_column('C:C', 10)
worksheet_result.set_column('D:D', 10)
worksheet_result.set_column('H:H', 10)
worksheet_result.set_column('I:I', 10)
worksheet_result.set_column('J:J', 10)
worksheet_result.set_column('K:K', 10)
worksheet_result.set_column('L:L', 10)
worksheet_result.set_column('M:M', 10)
worksheet_result.set_column('N:N', 10)
worksheet_result.set_column('O:O', 10)
worksheet_result.set_column('P:P', 10)    

worksheet_result.write(0, 0, "STK_CD", filter_format)
worksheet_result.write(0, 1, "STK_NM_KOR", filter_format)
worksheet_result.write(0, 2, "rcp_NO", filter_format)
worksheet_result.write(0, 3, "rcp_DT", filter_format)

worksheet_result.write(0, 4, "자산_양수도전_자산총계", filter_format)
worksheet_result.write(0, 5, "자산_증가감소_자산총계", filter_format)
worksheet_result.write(0, 6, "자산_양수도후_자산총계", filter_format)

worksheet_result.write(0, 7, "자산_양수도전_유동자산", filter_format)
worksheet_result.write(0, 8, "자산_증가감소_유동자산", filter_format)
worksheet_result.write(0, 9, "자산_양수도후_유동자산", filter_format)

worksheet_result.write(0, 10, "자산_양수도전_자본총계", filter_format)
worksheet_result.write(0, 11, "자산_증가감소_자본총계", filter_format)
worksheet_result.write(0, 12, "자산_양수도후_자본총계", filter_format)

worksheet_result.write(0, 13, "자산_양수도전_유동부채", filter_format)
worksheet_result.write(0, 14, "자산_증가감소_유동부채", filter_format)
worksheet_result.write(0, 15, "자산_양수도후_유동부채", filter_format)

worksheet_result.write(0, 16, "자산_양수도전_비유동부채", filter_format)
worksheet_result.write(0, 17, "자산_증가감소_비유동부채", filter_format)
worksheet_result.write(0, 18, "자산_양수도후_비유동부채", filter_format)

worksheet_result.write(0, 19, "url", filter_format)

for k in range(len(dart_div_list)):
    try: 
        STK_CD = dart_div_list[k].get('STK_CD')[0]
        STK_NM_KOR = dart_div_list[k].get('STK_NM_KOR')[0]
        rcp_NO = dart_div_list[k].get('rcp_NO')[0]
        rcp_DT = dart_div_list[k].get('rcp_DT')[0]
        url = dart_div_list[k].get('url')[0]
        
        자산_양수도전_자산총계 = dart_div_list[k].get('자산총계')[0]
        자산_증가감소_자산총계 = dart_div_list[k].get('자산총계')[1]
        자산_양수도후_자산총계 = dart_div_list[k].get('자산총계')[2]

        자산_양수도전_유동자산 = dart_div_list[k].get('유동자산')[0]
        자산_증가감소_유동자산 = dart_div_list[k].get('유동자산')[1]
        자산_양수도후_유동자산 = dart_div_list[k].get('유동자산')[2]

        자산_양수도전_자본총계 = dart_div_list[k].get('자본총계')[0]
        자산_증가감소_자본총계 = dart_div_list[k].get('자본총계')[1]
        자산_양수도후_자본총계 = dart_div_list[k].get('자본총계')[2]

        자산_양수도전_유동부채 = dart_div_list[k].get('유동부채')[0]
        자산_증가감소_유동부채 = dart_div_list[k].get('유동부채')[1]
        자산_양수도후_유동부채 = dart_div_list[k].get('유동부채')[2]

        자산_양수도전_비유동부채 = dart_div_list[k].get('비유동부채')[0]
        자산_증가감소_비유동부채 = dart_div_list[k].get('비유동부채')[1]
        자산_양수도후_비유동부채 = dart_div_list[k].get('비유동부채')[2]
        
    except ValueError:
        pass
    
    worksheet_result.write(k+1,0, STK_CD)
    worksheet_result.write(k+1,1, STK_NM_KOR)
    worksheet_result.write(k+1,2, rcp_DT)
    worksheet_result.write(k+1,3, rcp_NO)
    
    worksheet_result.write(k+1,4, 자산_양수도전_자산총계)
    worksheet_result.write(k+1,5, 자산_증가감소_자산총계)
    worksheet_result.write(k+1,6, 자산_양수도후_자산총계)
    
    worksheet_result.write(k+1,7, 자산_양수도전_유동자산)
    worksheet_result.write(k+1,8, 자산_증가감소_유동자산)
    worksheet_result.write(k+1,9, 자산_양수도후_유동자산)
    
    worksheet_result.write(k+1,10, 자산_양수도전_자본총계)
    worksheet_result.write(k+1,11, 자산_증가감소_자본총계)
    worksheet_result.write(k+1,12, 자산_양수도후_자본총계)
    
    worksheet_result.write(k+1,13, 자산_양수도전_유동부채)
    worksheet_result.write(k+1,14, 자산_증가감소_유동부채)
    worksheet_result.write(k+1,15, 자산_양수도후_유동부채)
    
    worksheet_result.write(k+1,16, 자산_양수도전_비유동부채)
    worksheet_result.write(k+1,17, 자산_증가감소_비유동부채)
    worksheet_result.write(k+1,18, 자산_양수도후_비유동부채)

    worksheet_result.write(k+1,19, url)
    
workbook.close()

In [62]:
print(div_trs)

[<tr>
<th align="CENTER" height="43" width="188">구      분</th>
<th align="CENTER" height="43" width="133">양수ㆍ도(전)<br/>(제102기 3분기)</th>
<th align="CENTER" height="43" width="133">증가(감소)</th>
<th align="CENTER" height="43" width="132">양수ㆍ도(후)</th>
</tr>, <tr>
<td height="20" valign="TOP" width="188">[유동자산]</td>
<td align="RIGHT" height="20" valign="TOP" width="133">809,433</td>
<td align="RIGHT" height="20" valign="TOP" width="133">41,633</td>
<td align="RIGHT" height="20" valign="TOP" width="132">851,066</td>
</tr>, <tr>
<td height="20" valign="TOP" width="188"> ㆍ당좌자산</td>
<td align="RIGHT" height="20" valign="TOP" width="133">400,638</td>
<td align="RIGHT" height="20" valign="TOP" width="133">361,500</td>
<td align="RIGHT" height="20" valign="TOP" width="132">762,128</td>
</tr>, <tr>
<td height="20" valign="TOP" width="188"> ㆍ재고자산</td>
<td align="RIGHT" height="20" valign="TOP" width="133">8,928</td>
<td align="RIGHT" height="20" valign="TOP" width="133">-</td>
<td align="RIGHT" height

# 페이지에서 재무제표 파싱

In [104]:
for i in range(len(df_sample)):
    target = df_sample.iloc[i]
#     print(target)
    crpCd = target.crp_cd
    crpNm = target.crp_nm
    rcpNo = target.rcp_no
    rcpDt = target.rcp_dt
    link = target.url 
    

In [88]:
# // 8
# treeNode1 = new Tree.TreeNode({
#     text: "Ⅶ. [합병등] 전ㆍ후의 요약재무정보",
#     id: "8",
#     cls: "text",
#     listeners: {
#         click: function() {viewDoc('20171113000213', '5840458', '8', '7805', '4194', 'dart3.xsd');}
#     }
# });
# cnt++;


# <iframe id="ifrm" frameborder="0" scrolling="auto" style="width: 100%; margin: 0px; padding: 0px; height: 891px;" title="tailFrame" src="/report/viewer.do?rcpNo=20171113000213&amp;dcmNo=5840458&amp;eleId=8&amp;offset=7805&amp;length=4194&amp;dtd=dart3.xsd"></iframe>

In [4]:
# 자산양수도 보고서 url 
# link= "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20190916900131"   #배당
link= "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20171113000213"  
# &dcmNo=5840458amp;eleId=8&amp;offset=7805&amp;length=4194&amp;dtd=dart3.xsd


handle = urllib.request.urlopen(link)
handle
data = handle.read()
soup2 = BeautifulSoup(data, 'html.parser', from_encoding='utf-8')  # http read
soup2


<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">

<html>
<head>
<title>
동화약품/합병등종료보고서/2017.11.13
</title>
<meta content="IE=EmulateIE8" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<style type="text/css">
    .x-window-dlg .ext-mb-download {
        background:transparent url(images/download.gif) no-repeat top left;
        height:46px;
    }
</style>
<link charset="utf-8" href="/css/viewer.css" rel="stylesheet" type="text/css"/>
<script src="/js/prototype.js" type="text/javascript"></script>
<script src="/js/jquery/jquery-all.js" type="text/javascript"></script>
<!-- 2014.03.11 ext 3.4 -->
<!--[if lte IE 8]><link rel="stylesheet" type="text/css" href="/js/ext-viewer/resources/css/ext-all-ie8.css" ><![endif]-->
<!--[if (gte IE 9)|!(IE)]><!--><link href="/js/ext-viewer/resources/css/ext-all.css" rel="stylesheet" type="text/css"/><!--<![endif]-->
<script src="/js/ext-viewer/adapt

In [5]:
test = soup2.find('a', {'href' : '#download'})['onclick']
print(test)

words = test.split("'")
print(words)

rcpNo= words[1]
dcmNo= words[3]
print(rcpNo)
print(dcmNo)

dart2 = soup2.find_all(string=re.compile('dart2.dtd'))
dart3 = soup2.find_all(string=re.compile('dart3.xsd'))
dart3

eleId = '8'  # 재무제표 8



# 뷰어 링크
if len(dart3) != 0:
    link2 = "http://dart.fss.or.kr/report/viewer.do?rcpNo=" + rcpNo + "&dcmNo=" + dcmNo + "&eleId=" + eleId + "&offset=4916&length=3668&dtd=dart3.xsd"
print(link2)

openPdfDownload('20171113000213', '5840458'); return false;
['openPdfDownload(', '20171113000213', ', ', '5840458', '); return false;']
20171113000213
5840458
http://dart.fss.or.kr/report/viewer.do?rcpNo=20171113000213&dcmNo=5840458&eleId=8&offset=4916&length=3668&dtd=dart3.xsd


In [6]:
handle = urllib.request.urlopen(link2)
#print(handle)
data = handle.read()
soup3 = BeautifulSoup(data, 'html.parser', from_encoding='utf-8')
# print(soup3)

tables = soup3.findAll("table")
tables  # list

[<table border="1" width="520">
 <colgroup>
 <col width="128"/>
 <col width="135"/>
 <col width="127"/>
 <col width="130"/>
 </colgroup>
 <tbody>
 <tr>
 <td align="CENTER" height="20" width="128">구    분</td>
 <td align="CENTER" height="20" width="135">자산양수도 전</td>
 <td align="CENTER" height="20" width="127">증가(감소)</td>
 <td align="CENTER" height="20" width="130">자산양수도 후</td>
 </tr>
 <tr>
 <td height="20" valign="TOP" width="128">자산</td>
 <td align="RIGHT" height="20" valign="TOP" width="135"><br/></td>
 <td align="RIGHT" height="20" valign="TOP" width="127"><br/></td>
 <td align="RIGHT" height="20" valign="TOP" width="130"><br/></td>
 </tr>
 <tr>
 <td height="20" valign="TOP" width="128">Ⅰ.유동자산</td>
 <td align="RIGHT" height="20" valign="TOP" width="135">162,257</td>
 <td align="RIGHT" height="20" valign="TOP" width="127">+85,350</td>
 <td align="RIGHT" height="20" valign="TOP" width="130">247,607</td>
 </tr>
 <tr>
 <td height="20" valign="TOP" width="128">Ⅱ.비유동자산</td>
 <td align="RIGH

In [7]:
if len(tables) == 1 or len(tables) == 2:
    div_table = soup3.find("table")
else:
    print("Tables", len(tables))
#     div_table = soup3.findAll("table")[2]
    div_table = soup3.findAll("table")[-1]
    
div_table  # 예외처리 후 한가지의 table 파싱

<table border="1" width="520">
<colgroup>
<col width="128"/>
<col width="135"/>
<col width="127"/>
<col width="130"/>
</colgroup>
<tbody>
<tr>
<td align="CENTER" height="20" width="128">구    분</td>
<td align="CENTER" height="20" width="135">자산양수도 전</td>
<td align="CENTER" height="20" width="127">증가(감소)</td>
<td align="CENTER" height="20" width="130">자산양수도 후</td>
</tr>
<tr>
<td height="20" valign="TOP" width="128">자산</td>
<td align="RIGHT" height="20" valign="TOP" width="135"><br/></td>
<td align="RIGHT" height="20" valign="TOP" width="127"><br/></td>
<td align="RIGHT" height="20" valign="TOP" width="130"><br/></td>
</tr>
<tr>
<td height="20" valign="TOP" width="128">Ⅰ.유동자산</td>
<td align="RIGHT" height="20" valign="TOP" width="135">162,257</td>
<td align="RIGHT" height="20" valign="TOP" width="127">+85,350</td>
<td align="RIGHT" height="20" valign="TOP" width="130">247,607</td>
</tr>
<tr>
<td height="20" valign="TOP" width="128">Ⅱ.비유동자산</td>
<td align="RIGHT" height="20" valign="TOP" w

### Ⅶ. [합병등] 전ㆍ후의 요약재무정보
[0] 자산  
[1] 1. Ⅰ.유동자산  
[2] 2. Ⅱ.비유동자산  
[3] 3. 자산총계  
[4] 부채  
[5] 4. Ⅰ.유동부채  
[6] 5. Ⅱ.비유동부채  
[7] 6. 부채총계  
[8] 자본  
[9] 7. Ⅰ.자본금  
[10] 8. Ⅱ.기타자본항목  
[11] 9. 자본총계  
[12] 10. 자산및부채총계  

In [9]:
div_trs = div_table.findAll('tr')
print(len(div_trs))
div_trs

14


[<tr>
 <td align="CENTER" height="20" width="128">구    분</td>
 <td align="CENTER" height="20" width="135">자산양수도 전</td>
 <td align="CENTER" height="20" width="127">증가(감소)</td>
 <td align="CENTER" height="20" width="130">자산양수도 후</td>
 </tr>, <tr>
 <td height="20" valign="TOP" width="128">자산</td>
 <td align="RIGHT" height="20" valign="TOP" width="135"><br/></td>
 <td align="RIGHT" height="20" valign="TOP" width="127"><br/></td>
 <td align="RIGHT" height="20" valign="TOP" width="130"><br/></td>
 </tr>, <tr>
 <td height="20" valign="TOP" width="128">Ⅰ.유동자산</td>
 <td align="RIGHT" height="20" valign="TOP" width="135">162,257</td>
 <td align="RIGHT" height="20" valign="TOP" width="127">+85,350</td>
 <td align="RIGHT" height="20" valign="TOP" width="130">247,607</td>
 </tr>, <tr>
 <td height="20" valign="TOP" width="128">Ⅱ.비유동자산</td>
 <td align="RIGHT" height="20" valign="TOP" width="135">170,261</td>
 <td align="RIGHT" height="20" valign="TOP" width="127">-30,450</td>
 <td align="RIGHT" heigh

In [10]:
# 단위찾기
단위 = soup3.findAll('p')[2].text.strip()
print(단위) 

(단위 ; 백만원)


### column name
[0] 구분  
[1] 자산양수도 전  
[2] 증가(감소)  
[3] 자산양수도 후  

In [11]:
dart_div_list = list()
dart_div_sublist = list()

In [49]:
def cleansing(text):
    pattern = r'quot' #일본어,한자제거  quot
    text = re.sub(pattern = pattern, repl='', string = text)
    pattern = r'([ぁ-ゟ゠-ヿ一-鿋])' #일본어,한자제거  quot
    text = re.sub(pattern = pattern, repl='', string = text)
    pattern= r'\xa0'  #줄바꿈제거
    text = re.sub(pattern = pattern, repl='', string = text)
    pattern= r'URL 복사 이웃추가 본문 기타 기능 번역보기'
    text = re.sub(pattern = pattern, repl='', string = text)
    pattern= r'([ㄱ-ㅎㅏ-ㅣ])' # 한글자음 제거
    text = re.sub(pattern = pattern, repl='', string = text)
    pattern= r'[^\w\s]' # 특수기호 제거
    text = re.sub(pattern = pattern, repl='', string = text)
    pattern= r'[\Ⅰ\Ⅱ\ㆍ]' # 로마자 제거
    text = re.sub(pattern = pattern, repl='', string = text)
    pattern= r' ' # 띄어쓰기 제거
    text = re.sub(pattern = pattern, repl='', string = text)
    
    return text

In [13]:
# 유동자산 2
div_trs[2]
print(div_trs[2])

# 열 이름 찾기
div_trs[2].findAll('td')
print(div_trs[2].findAll('td'))

div_trs[2].findAll('td')[0].text.strip()
# div_trs[2].findAll('td')[1].text.strip()
# div_trs[2].findAll('td')[2].text.strip()
# div_trs[2].findAll('td')[3].text.strip()

<tr>
<td height="20" valign="TOP" width="128">Ⅰ.유동자산</td>
<td align="RIGHT" height="20" valign="TOP" width="135">162,257</td>
<td align="RIGHT" height="20" valign="TOP" width="127">+85,350</td>
<td align="RIGHT" height="20" valign="TOP" width="130">247,607</td>
</tr>
[<td height="20" valign="TOP" width="128">Ⅰ.유동자산</td>, <td align="RIGHT" height="20" valign="TOP" width="135">162,257</td>, <td align="RIGHT" height="20" valign="TOP" width="127">+85,350</td>, <td align="RIGHT" height="20" valign="TOP" width="130">247,607</td>]


'Ⅰ.유동자산'

In [14]:
from collections import defaultdict

dict1= defaultdict(list)

# 단위찾기
단위 = soup3.findAll('p')[2].text.strip()
print(단위)
dict1['단위'].append(단위)
print(dict(dict1))

# 딕셔너리에 넣기
for i in range(len(div_trs)):
    for j in range(1,4):
        key = cleansing(div_trs[i].findAll('td')[0].text.strip())
        val = cleansing(div_trs[i].findAll('td')[j].text.strip())
        dict1[key].append(val)
dict(dict1)

(단위 ; 백만원)
{'단위': ['(단위 ; 백만원)']}


{'단위': ['(단위 ; 백만원)'],
 '구분': ['자산양수도전', '증가감소', '자산양수도후'],
 '자산': ['', '', ''],
 '유동자산': ['162257', '85350', '247607'],
 '비유동자산': ['170261', '30450', '139811'],
 '자산총계': ['332518', '54900', '387418'],
 '부채': ['', '', ''],
 '유동부채': ['57216', '350', '57566'],
 '비유동부채': ['18546', '', '18546'],
 '부채총계': ['75762', '350', '76112'],
 '자본': ['', '', ''],
 '자본금': ['27931', '', '27931'],
 '기타자본항목': ['228825', '54550', '283375'],
 '자본총계': ['256756', '54550', '311306'],
 '자산및부채총계': ['332518', '54900', '387418']}

In [16]:
dart_div_list.append(dict(dict1))
dart_div_list

[{'단위': ['(단위 ; 백만원)'],
  '구분': ['자산양수도전', '증가감소', '자산양수도후'],
  '자산': ['', '', ''],
  '유동자산': ['162257', '85350', '247607'],
  '비유동자산': ['170261', '30450', '139811'],
  '자산총계': ['332518', '54900', '387418'],
  '부채': ['', '', ''],
  '유동부채': ['57216', '350', '57566'],
  '비유동부채': ['18546', '', '18546'],
  '부채총계': ['75762', '350', '76112'],
  '자본': ['', '', ''],
  '자본금': ['27931', '', '27931'],
  '기타자본항목': ['228825', '54550', '283375'],
  '자본총계': ['256756', '54550', '311306'],
  '자산및부채총계': ['332518', '54900', '387418']}]

In [24]:
try: 
    단위 = dart_div_list[0].get('단위')[0]
    자산_양수도전_자산총계 = dart_div_list[0].get('자산총계')[0]
    자산_증가감소_자산총계 = dart_div_list[0].get('자산총계')[1]
    자산_양수도후_자산총계 = dart_div_list[0].get('자산총계')[2]

    자산_양수도전_유동자산 = dart_div_list[0].get('유동자산')[0]
    자산_증가감소_유동자산 = dart_div_list[0].get('유동자산')[1]
    자산_양수도후_유동자산 = dart_div_list[0].get('유동자산')[2]

    자산_양수도전_자본총계 = dart_div_list[0].get('자본총계')[0]
    자산_증가감소_자본총계 = dart_div_list[0].get('자본총계')[1]
    자산_양수도후_자본총계 = dart_div_list[0].get('자본총계')[2]

    자산_양수도전_유동부채 = dart_div_list[0].get('유동부채')[0]
    자산_증가감소_유동부채 = dart_div_list[0].get('유동부채')[1]
    자산_양수도후_유동부채 = dart_div_list[0].get('유동부채')[2]

    자산_양수도전_비유동부채 = dart_div_list[0].get('비유동부채')[0]
    자산_증가감소_비유동부채 = dart_div_list[0].get('비유동부채')[1]
    자산_양수도후_비유동부채 = dart_div_list[0].get('비유동부채')[2]
except ValueError:
    pass
    

## 엑셀 파일에 만들기 

In [81]:
cur_dir = os.getcwd()
cur_dir

'C:\\Users\\HS\\Desktop'

In [85]:
cur_dir = os.getcwd()
cur_dir

workbook_name = "DART_dividends_baseline.xlsx"

workbook = xlsxwriter.Workbook(workbook_name)

worksheet_result = workbook.add_worksheet('result')
filter_format = workbook.add_format({'bold':True,
                                    'fg_color': '#D7E4BC'
                                    })

percent_format = workbook.add_format({'num_format': '0.00%'})

roe_format = workbook.add_format({'bold':True,
                                  'underline': True,
                                  'num_format': '0.00%'})

num_format = workbook.add_format({'num_format':'0.00'})
num2_format = workbook.add_format({'num_format':'#,##0'})
num3_format = workbook.add_format({'num_format':'#,##0.00',
                                  'fg_color':'#FCE4D6'})

worksheet_result.set_column('A:A', 10)
worksheet_result.set_column('B:B', 10)
worksheet_result.set_column('C:C', 10)
worksheet_result.set_column('D:D', 10)
worksheet_result.set_column('H:H', 10)
worksheet_result.set_column('I:I', 10)
worksheet_result.set_column('J:J', 10)
worksheet_result.set_column('K:K', 10)
worksheet_result.set_column('L:L', 10)
worksheet_result.set_column('M:M', 10)
worksheet_result.set_column('N:N', 10)
worksheet_result.set_column('O:O', 10)
worksheet_result.set_column('P:P', 10)

0

In [86]:

worksheet_result.write(0, 1, "자산_양수도전_자산총계", filter_format)
worksheet_result.write(0, 2, "자산_증가감소_자산총계", filter_format)
worksheet_result.write(0, 3, "자산_양수도후_자산총계", filter_format)

worksheet_result.write(0, 4, "자산_양수도전_유동자산", filter_format)
worksheet_result.write(0, 5, "자산_증가감소_유동자산", filter_format)
worksheet_result.write(0, 6, "자산_양수도후_유동자산", filter_format)

worksheet_result.write(0, 7, "자산_양수도전_자본총계", filter_format)
worksheet_result.write(0, 8, "자산_증가감소_자본총계", filter_format)
worksheet_result.write(0, 9, "자산_양수도후_자본총계", filter_format)

worksheet_result.write(0, 10, "자산_양수도전_유동부채", filter_format)
worksheet_result.write(0, 11, "자산_증가감소_유동부채", filter_format)
worksheet_result.write(0, 12, "자산_양수도후_유동부채", filter_format)

worksheet_result.write(0, 13, "자산_양수도전_비유동부채", filter_format)
worksheet_result.write(0, 14, "자산_증가감소_비유동부채", filter_format)
worksheet_result.write(0, 15, "자산_양수도후_비유동부채", filter_format)


0

In [87]:
for k in range(len(dart_div_list)):

    worksheet_result.write(k+1,1, 자산_양수도전_자산총계)
    worksheet_result.write(k+1,2, 자산_증가감소_자산총계)
    worksheet_result.write(k+1,3, 자산_양수도후_자산총계)
    
    worksheet_result.write(k+1,4, 자산_양수도전_유동자산)
    worksheet_result.write(k+1,5, 자산_증가감소_유동자산)
    worksheet_result.write(k+1,6, 자산_양수도후_유동자산)
    
    worksheet_result.write(k+1,7, 자산_양수도전_자본총계)
    worksheet_result.write(k+1,8, 자산_증가감소_자본총계)
    worksheet_result.write(k+1,9, 자산_양수도후_자본총계)
    
    worksheet_result.write(k+1,10, 자산_양수도전_유동부채)
    worksheet_result.write(k+1,11, 자산_증가감소_유동부채)
    worksheet_result.write(k+1,12, 자산_양수도후_유동부채)
    
    worksheet_result.write(k+1,13, 자산_양수도전_비유동부채)
    worksheet_result.write(k+1,14, 자산_증가감소_비유동부채)
    worksheet_result.write(k+1,15, 자산_양수도후_비유동부채)


In [88]:
workbook.close()